### ingest data

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../data/data_preprocessed.parquet', engine='pyarrow')
data.head(3)

,id,topic,question_title,question_content,best_answer,topic_name
0,0,4,why doesn t an optical mouse work on a glass t...,or even on some surfaces?,Optical mice use an LED and a camera to rapidl...,Computers & Internet
1,1,5,what is the best off road motorcycle trail,long-distance trail throughout CA,i hear that the mojave road is amazing!<br />\...,Sports
2,2,2,what is trans fat how to reduce that,I heard that tras fat is bad for the body. Wh...,Trans fats occur in manufactured foods during ...,Health


### setup

vectorizer

In [3]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
count_vectorizer = CountVectorizer()
count_vectorizer

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"
,analyzer,'word'


tfidf

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer

In [6]:
tfidf = TfidfTransformer(use_idf=True, norm="l2", smooth_idf=True)
tfidf

,norm,'l2'
,use_idf,True
,smooth_idf,True
,sublinear_tf,False


tokenizer

In [7]:
def tokenizer(text):
    return text.split()

stemming

In [8]:
from nltk.stem.porter import PorterStemmer

In [9]:
porter_stemming = PorterStemmer()
porter_stemming

<PorterStemmer>

In [10]:
def tokenizer_porter(text):
    return [porter_stemming.stem(word) for word in text.split()]

stopwords

In [11]:
%%capture
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Seu
[nltk_data]     Computador\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
from nltk.corpus import stopwords


stop = stopwords.words("english")
stop[:3]

['a', 'about', 'above']

### training model

split data

In [13]:
from sklearn.model_selection import train_test_split


X = data["question_title"]
y = data["topic"]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [14]:
print(X_train.shape, y_train.shape)

(560000,) (560000,)


In [15]:
print(X_test.shape, y_test.shape)

(140000,) (140000,)


build model

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [17]:
tfidf = TfidfVectorizer(
    strip_accents=None, lowercase=False, preprocessor=None
)

tfidf

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,False
,preprocessor,None
,tokenizer,None
,analyzer,'word'
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"


In [18]:
small_param_grid = [{'vect__ngram_range': [(1, 1)],
                     'vect__stop_words': [None],
                     'vect__tokenizer': [tokenizer, tokenizer_porter],
                     'clf__penalty': ['l2'],
                     'clf__C': [1.0, 10.0]},
                    {'vect__ngram_range': [(1, 1)],
                     'vect__stop_words': [stop, None],
                     'vect__tokenizer': [tokenizer],
                     'vect__use_idf':[False],
                     'vect__norm':[None],
                     'clf__penalty': ['l2'],
                  'clf__C': [1.0, 10.0]},
              ]


small_param_grid[0]

{'vect__ngram_range': [(1, 1)],
 'vect__stop_words': [None],
 'vect__tokenizer': [<function __main__.tokenizer(text)>,
  <function __main__.tokenizer_porter(text)>],
 'clf__penalty': ['l2'],
 'clf__C': [1.0, 10.0]}

In [19]:
estimator = Pipeline(
    [
        ('vect', tfidf), ('clf', LogisticRegression(solver='liblinear'))
    ]
)

estimator

,steps,"[('vect', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,False
,preprocessor,None
,tokenizer,None


In [20]:
grid_search = GridSearchCV(
    estimator, small_param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1
)

grid_search

,estimator,Pipeline(step...liblinear'))])
,param_grid,"[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'], 'vect__ngram_range': [(1, ...)], 'vect__stop_words': [None], ...}, {'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'], 'vect__ngram_range': [(1, ...)], 'vect__norm': [None], ...}]"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [21]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


c:\Users\Seu Computador\Documents\Github\yahooTagger\venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Seu Computador\Documents\Github\yahooTagger\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


,estimator,Pipeline(step...liblinear'))])
,param_grid,"[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'], 'vect__ngram_range': [(1, ...)], 'vect__stop_words': [None], ...}, {'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'], 'vect__ngram_range': [(1, ...)], 'vect__norm': [None], ...}]"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [22]:
print(f'Best parameter set: {grid_search.best_params_}')

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x000001DDC2D05BC0>}


In [23]:
print(f'CV Accuracy: {grid_search.best_score_:.2f}')

CV Accuracy: 0.64


In [24]:
best_estimator = grid_search.best_estimator_
best_estimator

,steps,"[('vect', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,False
,preprocessor,None
,tokenizer,<function tok...001DDC2D05BC0>


In [25]:
predictions = best_estimator.predict(X_test)
predictions[:3]

array([8, 8, 9])

In [41]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
accuracy

0.6477928571428572

mlflow

In [28]:
from mlflow.models import infer_signature

In [29]:
signature = infer_signature(X_train, y_train)
signature

c:\Users\Seu Computador\Documents\Github\yahooTagger\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


inputs: 
  ['question_title': string (required)]
outputs: 
  ['topic': long (required)]
params: 
  None

In [ ]:
import pickle


with open("../model/model.pkl", "wb") as f:
    pickle.dump(best_estimator, f)

In [ ]:
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("initial experiment")

with mlflow.start_run():
    mlflow.log_params(grid_search.best_params_) 
    mlflow.log_metric("accuracy", accuracy)

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(best_estimator, "model", registered_model_name="Best RandomForest Model")
    else:
        mlflow.sklearn.log_model(best_estimator, "model")


2025/11/05 23:59:11 INFO mlflow.tracking.fluent: Experiment with name 'initial experiment' does not exist. Creating a new experiment.
2025/11/05 23:59:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
